<h1 align="center">
 Obter Catálogo de Teses e Dissertações <br/>
 <img src="https://dadosabertos.capes.gov.br/img/caixa.png"  alt="Dados Capes"/>
</h1>

No portal de dados abertos da *CAPES*, há dados sobre os procedimentos realizados e informações sobre os programas de pós-graduação do Brasil. Parte dos dados são fornecidos por coordenadores desses programas através do módulo **Coleta CAPES** disponível na [Plataforma Sucupira](https://sucupira.capes.gov.br/sucupira/).

Dentre esses dados, estamos interessados no **Catálogo de Teses e Dissertações** que contém informações sobre as teses e dissertações defendidas nos programas de pós-graduação. Esses dados estão separados em períodos de tempo, atualmente há os seguintes conjuntos: 

- [[1987 a 2012] Catálogo de Teses e Dissertações - Brasil](https://dadosabertos.capes.gov.br/dataset/1987-a-2012-catalogo-de-teses-e-dissertacoes-brasil)
- [[2013 a 2016] Catálogo de Teses e Dissertações - Brasil](https://dadosabertos.capes.gov.br/dataset/catalogo-de-teses-e-dissertacoes-de-2013-a-2016)
- [[2017 a 2020] Catálogo de Teses e Dissertações - Brasil](https://dadosabertos.capes.gov.br/dataset/2017-2020-catalogo-de-teses-e-dissertacoes-da-capes)
- [[2021 a 2024] Catálogo de Teses e Dissertações - Brasil](https://dadosabertos.capes.gov.br/dataset/2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil)

Nosso objetivo é realizar o download dessas tabelas, unir e padronizar as colunas de dados em comum. 

## Acessando os dados

Podemos consultar esses dados através da [API do CKAN](https://docs.ckan.org/en/latest/api/index.html), por exemplo, a URL abaixo lista todos os grupos disponíveis:

https://dadosabertos.capes.gov.br/api/3/action/group_list

Para acessar informações de um grupo específico, como teses e dissertações, usamos o `group_show?id=identificador-grupo` e  `include_datasets=true` para listar as base de dados.

https://dadosabertos.capes.gov.br/api/3/action/group_show?id=catalogo-de-teses-e-dissertacoes-brasil&include_datasets=true


Para compreender mais como usar o *CKAN API*, acesse o [API guide](https://docs.ckan.org/en/latest/api/index.html).

In [ ]:
import requests
import pandas as pd
from tqdm.auto import tqdm
from pathlib import Path

In [ ]:
# diretório onde será salvo os dados
workdir = Path("../data")

# cria o diretório se não existir
if not workdir.exists():
    workdir.mkdir()

### Download dos dados

Para acessar a resposta da _CKAN API_ utilizamos a biblioteca `requests` e extraímos a variável _result_.

In [ ]:
# realizar a requisição HTTP
response = requests.get(
    "https://dadosabertos.capes.gov.br/api/3/action/package_search?",
    params={
        "q": "groups:catalogo-de-teses-e-dissertacoes-brasil",
        "fl": "id,name,metadata_created,metadata_modified, res_url",
    },
)

# se a requisiçao foi realizada com sucesso (status code = 200)
assert response.status_code == requests.codes.ok

# atribuir json da resposta a um dicionário
response_dict = response.json()

# verificar se a requisição foi bem sucedida
assert response_dict["success"] is True

# extrair os resultados
results = response_dict.get("result", {}).get("results", [])

No _json_ retornado, estamos interessados no campo `res_url` que contém o link para baixar os dados.  Extraímos os links que estão em formato `.csv` para realizar o download.

In [ ]:
csv_urls = [
    url
    for result in results
    for url in result.get("res_url", [])
    if url.endswith(".csv")
]

print(f"Encontrados {len(csv_urls)} arquivos CSV")

Usamos a biblioteca `requests` para realizar o download dos arquivos, o código abaixo realiza o download dos arquivos e salva na pasta `data`. O processo é demorado e varia de acordo com a velocidade da internet e tamanho dos arquivos. Alguns erros podem ocorrer, como por exemplo, o servidor não responder ou a conexão cair. Nesses casos, basta rodar o código novamente que ele continua o download do arquivo que parou.

In [ ]:
for url in tqdm(csv_urls):

    filename = url.split("/")[-1]
    filepath = workdir / filename

    if filepath.exists():
        print(f"Arquivo {filename} já existe em {filepath}")
        continue

    response = requests.get(url)
    assert response.status_code == requests.codes.ok

    with open(filepath, "wb") as f:
        f.write(response.content)

    print(f"Arquivo {filename} salvo com sucesso em {filepath}")

### Agrupar dados por conjunto

Há quatro grupos de dados de teses e dissertações como mencionamos no começo desse notebook. Ao analisarmos os metadados contido no portal de dados abertos observamos que os conjuntos de 2013 a 2016, 2017 a 2020 e 2021 a 2024 possuem o mesmo tipo de dados. Com base nisso, vamos agrupar os arquivos CSVs em dois grandes arquivos:

- `catalogo-de-teses-e-dissertacoes-de-1987-a-2012.parquet` - contempla todos os dados coletados no período de 1987 a 2012
- `catalogo-de-teses-e-dissertacoes-de-2013-a-2024.parquet` - contempla todos os dados coletados no período de 2013 a 2024

In [ ]:
files_list = list(workdir.glob("*.csv"))

thesis_before_2013 = pd.concat(
    pd.read_csv(file, sep=";", encoding="latin1", low_memory=False)
    for file in files_list
    if file.name.startswith("dados_")
)

print(f"Total de registros antes de 2013: {thesis_before_2013.shape[0]:,}")

thesis_since_2013 = pd.concat(
    pd.read_csv(file, sep=";", encoding="latin1", low_memory=False)
    for file in files_list
    if not file.name.startswith("dados_")
)

print(f"Total de registros a partir de 2013: {thesis_since_2013.shape[0]:,}")

Antes de salvar como *parquet*, fazemos um pré-tratamento nas colunas `CD_SUBAREA_CONHECIMENTO` e `CD_ESPECIALIDADE`. Essas colunas tem cadastros com valor "NI" que provavelmente significa "Não Informado" e valores preenchidos com nulos. Inicialmente, vamos apenas substituir os valores nulos por *strings* vazias.

In [ ]:
thesis_before_2013.to_parquet(
    workdir / "catalogo-de-teses-e-dissertacoes-de-1987-a-2012"
)
thesis_since_2013["CD_SUBAREA_CONHECIMENTO"] = (
    thesis_since_2013["CD_SUBAREA_CONHECIMENTO"].fillna("").astype(str)
)
thesis_since_2013["CD_ESPECIALIDADE"] = (
    thesis_since_2013["CD_ESPECIALIDADE"].fillna("").astype(str)
)
thesis_since_2013.to_parquet(
    workdir / "catalogo-de-teses-e-dissertacoes-de-2013-a-2024.parquet"
)

## Padronização de colunas

As informações armazenadas antes de 2013 e a partir de 2013 sobre as teses e dissertações possuem padrões de colunas diferentes, vamos selecionar apenas as colunas em comum e renomeá-las para possuírem os mesmos nomes. Com base nas informações nos metadados definimos as seguintes informações em comum:


- AN_BASE
- CD_PROGRAMA
- NM_REGIAO
- SG_UF_IES
- SG_ENTIDADE_ENSINO
- NM_ENTIDADE_ENSINO
- NM_PROGRAMA
- CD_GRANDE_AREA_CONHECIMENTO
- NM_GRANDE_AREA_CONHECIMENTO
- CD_AREA_CONHECIMENTO
- NM_AREA_CONHECIMENTO
- NM_AREA_AVALIACAO
- NM_DISCENTE
- NM_PRODUCAO
- NM_GRAU_ACADEMICO
- DS_PALAVRA_CHAVE
- NR_VOLUME
- NR_PAGINAS
- NM_IDIOMA
- DS_RESUMO
- NM_LINHA_PESQUISA
- DS_URL_TEXTO_COMPLETO

In [ ]:
common = [
    "AN_BASE",
    "CD_PROGRAMA",
    "NM_REGIAO",
    "SG_UF_IES",
    "SG_ENTIDADE_ENSINO",
    "NM_ENTIDADE_ENSINO",
    "NM_PROGRAMA",
    "CD_GRANDE_AREA_CONHECIMENTO",
    "NM_GRANDE_AREA_CONHECIMENTO",
    "CD_AREA_CONHECIMENTO",
    "NM_AREA_CONHECIMENTO",
    "NM_AREA_AVALIACAO",
    "NM_DISCENTE",
    "NM_PRODUCAO",
    "NM_GRAU_ACADEMICO",
    "DS_PALAVRA_CHAVE",
    "NR_VOLUME",
    "NR_PAGINAS",
    "NM_IDIOMA",
    "DS_RESUMO",
    "NM_LINHA_PESQUISA",
    "DS_URL_TEXTO_COMPLETO",
]

thesis_since_2013 = thesis_since_2013[common]

In [ ]:
thesis_before_2013.rename(
    columns={
        "AnoBase": "AN_BASE",
        "CodigoPrograma": "CD_PROGRAMA",
        "Regiao": "NM_REGIAO",
        "Uf": "SG_UF_IES",
        "SiglaIes": "SG_ENTIDADE_ENSINO",
        "NomeIes": "NM_ENTIDADE_ENSINO",
        "NomePrograma": "NM_PROGRAMA",
        "GrandeAreaCodigo": "CD_GRANDE_AREA_CONHECIMENTO",
        "GrandeAreaDescricao": "NM_GRANDE_AREA_CONHECIMENTO",
        "AreaConhecimentoCodigo": "CD_AREA_CONHECIMENTO",
        "AreaConhecimento": "NM_AREA_CONHECIMENTO",
        "AreaAvaliacao": "NM_AREA_AVALIACAO",
        "Autor": "NM_DISCENTE",
        "TituloTese": "NM_PRODUCAO",
        "Nivel": "NM_GRAU_ACADEMICO",
        "PalavrasChave": "DS_PALAVRA_CHAVE",
        "Volume": "NR_VOLUME",
        "NumeroPaginas": "NR_PAGINAS",
        "Idioma": "NM_IDIOMA",
        "ResumoTese": "DS_RESUMO",
        "LinhaPesquisa": "NM_LINHA_PESQUISA",
        "URLTextoCompleto": "DS_URL_TEXTO_COMPLETO",
    },
    inplace=True,
)

thesis_before_2013 = thesis_before_2013[common]

Por fim, realizamos o merge de ambos os conjuntos de dados em um grande arquivo parquet chamado `catalogo-de-teses-e-dissertacoes.parquet`.

In [ ]:
thesis = pd.concat([thesis_before_2013, thesis_since_2013], ignore_index=True)

In [ ]:
thesis["NR_VOLUME"] = (
    thesis["NR_VOLUME"].fillna("").astype(str)
)  # preencher valores nulos com string vazia
thesis.to_parquet(workdir / "catalogo-de-teses-e-dissertacoes.parquet")